In [41]:
import json

In [1]:
from train_retriever import *

/home/wangjunh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/wangjunh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/wangjunh/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/wangjunh/anaconda3/lib/python3.7/site-packages/tensorflow/python/frame

In [2]:
encoder_question = BertEncoder(bert_question, max_question_len_global)
encoder_paragarph = BertEncoder(bert_paragraph, max_paragraph_len_global)
ret = Retriver(encoder_question, encoder_paragarph, tokenizer)

checkpoint_callback = ModelCheckpoint(
    filepath='out/{epoch}-{val_loss:.2f}-{val_acc:.2f}',
    save_top_k=10,
    verbose=True,
    monitor='val_acc',
    mode='max'
)

early_stopping = EarlyStopping('val_acc', mode='max')

trainer = pl.Trainer(
    gpus=8,
    distributed_backend='dp',
    val_check_interval=0.000001,
    min_epochs=1,
    checkpoint_callback=checkpoint_callback,
    early_stop_callback=early_stopping,
    gradient_clip_val=0.5
)
ret_trainee = RetriverTrainer(ret)

In [49]:
tmp = torch.load('out/epoch=0-val_loss=4.66-val_acc=0.37.ckpt')

In [52]:
ret_trainee.load_state_dict(tmp['state_dict'])

<All keys matched successfully>

In [53]:
q = 'Should I practice social distancing?'

s3 = {'section': 'Restaurants, businesses and public spaces', 'content': ['The Gouvernement du Québec\xa0is ordering until March\xa030,\xa02020 the closing of the following businesses and public spaces:', 'aquariums;skating rinks;bars and discotheques;libraries;sugar shacks;training gyms, dance halls and spinning, zumba and yoga centres;indoor soccer arenas;outdoor recreation centres;cinemas and games arcades;sample counters and booths in grocery and department stores;recreational sites (ski resorts, amusement parks, trampoline centres, etc.);museums;water parks;swimming pools, spas and saunas;restaurants offering buffet service;performance spaces;theatres;zoos;all other similar facilities.', 'The Gouvernement du Québec is asking restaurants and coffee shop owners to limit the number of customers to 50% of the rooms’ capacity, that is, one table in two. However, buffet-style restaurants and sugar shacks must close temporarily.', 'Take-out orders, deliveries and drive-through services are permitted.', 'The\xa0SQDC and SAQ\xa0remain open, but the number of clients must be limited in order to respect social distancing. Online orders are encouraged.', 'For\xa0places of worship, religious gatherings of all faiths should be suspended, unless they are essential. If a gathering takes place, social distancing must be observed.'], 'file_name': '1_en.json'} 

s2 = {'section': 'What resources are available for First Nations and Inuit?', 'content': ['Gouvernement du Québec', 'First Nations of Quebec and Labrador Health and Social Services Commission (FNQLHSSC)\xa0', 'Nunavik Regional Board of Health and Social Services\xa0', 'Cree Board of Health and Social Services of James Bay\xa0', 'We also encourage you to refer to your local health authorities.'], 'file_name': '4_en.json'} 

s1 = {'section': 'You can contact the Info-Social 811 service (24/7 psychosocial telephone consultation)', 'content': ['Contact the\xa0Info-Social\xa0811 service\xa0if you are experiencing:', 'stress or anxiety;persistent emotional reactions (frequent crying, irritability, and so on);panic;invasive, frightening ideas that you are constantly turning over in your mind;social repercussions of the disease.', 'Psychosocial intervention professionals will offer you support and provide information and advice according to your needs.'], 'file_name': '1_en.json'} 

In [54]:
s1 = ' '.join(s1['content'])
s2 = ' '.join(s2['content'])
s3 = ' '.join(s3['content'])

In [55]:
ret_trainee.retriever.predict(q, [s1, s2, s3] * 3000)

(['The Gouvernement du Québec\xa0is ordering until March\xa030,\xa02020 the closing of the following businesses and public spaces: aquariums;skating rinks;bars and discotheques;libraries;sugar shacks;training gyms, dance halls and spinning, zumba and yoga centres;indoor soccer arenas;outdoor recreation centres;cinemas and games arcades;sample counters and booths in grocery and department stores;recreational sites (ski resorts, amusement parks, trampoline centres, etc.);museums;water parks;swimming pools, spas and saunas;restaurants offering buffet service;performance spaces;theatres;zoos;all other similar facilities. The Gouvernement du Québec is asking restaurants and coffee shop owners to limit the number of customers to 50% of the rooms’ capacity, that is, one table in two. However, buffet-style restaurants and sugar shacks must close temporarily. Take-out orders, deliveries and drive-through services are permitted. The\xa0SQDC and SAQ\xa0remain open, but the number of clients must 